In [1]:
import numpy as np
import copy
import pandas as pd
from skimage.feature import peak_local_max
import os
import copy

%matplotlib inline
import matplotlib.pyplot as plt

#CODATA 2017
kB = 1.38064903E-23  #J/K
Na = 6.022140758E+23 #1/mol

In [2]:
plt.rcParams.update({'font.size': 16})
from matplotlib.ticker import FormatStrFormatter

In [3]:
import feasst
import lnPi

In [44]:
#import feasst
#import lnPi
#import os
#import copy

#function to tag 'LD' and 'HD' phases
def tag_phases(x):
    if x.base.num_phases_max !=2:
        raise ValueError('bad tag function')
        

    if x.nphase==1:
        if x[0].density<0.5:
            return np.array([0])
        else:
            return np.array([1])
    
    elif x.nphase==2:
        return np.argsort(-x.argmax[0])
        return np.array([0,1])
    else:
        raise ValueError('bad nphase')

def append_data(N0,array0,N1,array1,smooth_type="mean"):
    #Splices array1 onto array0, with smoothing
    N_out = [x for x in N0]
    array_out = [x for x in array0]
    ref_min = N1[0]
    ref_max = N0[-1]
    for Ni in N1[1:]:  #start with the second position
        if Ni <= ref_max:
            if smooth_type == "mean":
                delta = 0.5*( (array0[Ni]-array0[Ni-1]) + (array1[Ni-ref_min]-array1[Ni-ref_min-1])  )
            else:
                raise Exception("Unknown smoothing type: "+smooth_type)
            array_out[Ni] = array_out[Ni-1]+delta
        else:
            delta = (array1[Ni-ref_min]-array1[Ni-ref_min-1])
            array_out.append(array_out[-1]+delta)
            N_out.append(N_out[-1]+1)
    return N_out, array_out

class feasst_analysis(lnPi.lnPi_phases):
    def __init__(self,
                 base,
                 phases='get',
                 argmax='get',
                 argmax_kwargs=None,
                 phases_kwargs=None,
                 build_kwargs=None,
                 ftag_phases=None,
                 ftag_phases_kwargs=None):
        super(feasst_analysis, self).__init__(base,
                                              phases=phases,
                                              argmax=argmax,
                                              argmax_kwargs=argmax_kwargs,
                                              phases_kwargs=phases_kwargs,
                                              build_kwargs=build_kwargs,
                                              ftag_phases=ftag_phases,
                                              ftag_phases_kwargs=ftag_phases_kwargs)
        self.energy = np.array([]) ## HOW DO WE SHAPE THIS?
        #self.energy2 = np.array([]) ## HOW DO WE SHAPE THIS?
        self.Sx = np.array([])
        self.canSx = False
        self.extrapolated = False
        
    @classmethod
    def from_restart(cls,
                     source,
                     prefix):
        #Constructor method to build object from FEASST restart file(s)
        
        #Determine restart file structure/pattern
        input_files = [ x for x in os.listdir(source) if (prefix in x and "criteria" in x and "bak" not in x and "rng" not in x) ]
        windows = len(input_files)
                
        # Stitch the windows together
        for window in range(windows):
            file_stub = source+prefix+str(window)
        
            space = feasst.Space(file_stub+"space")
            boxLength = [space.boxLength(0),space.boxLength(1),space.boxLength(2)]
            criteria = feasst.CriteriaWLTMMC(file_stub+"criteria")
            beta = criteria.beta()
            lnZ = np.log(criteria.activ(0))
            volume = boxLength[0]*boxLength[1]*boxLength[2]
            nMin = int(np.ceil(criteria.mMin()))
            nMax = int(np.floor(criteria.mMax()))
    
            N_w = [x for x in range(nMin,nMax+1)]
            lnPi_w = [x for x in np.array(criteria.lnPIdouble())]
            bins = len(lnPi_w)
            energy_w = [ criteria.pe(i).sumDble()/criteria.pe(i).nValues() if criteria.pe(i).nValues()!=0 else 0. for i in range(0,bins)]
            #energy2_w = [ criteria.pe(i).sumSqDble()/criteria.pe(i).nValues() if criteria.pe(i).nValues()!=0 else 0. for i in range(0,bins)]

            # Return FEASST data as Python lists to enable easier appending
            if window == 0:
                # Deep copy first window into master arrays
                N = copy.deepcopy(N_w)
                lnPi = copy.deepcopy(lnPi_w)
                energy = copy.deepcopy(energy_w)
                #energy2 = copy.deepcopy(energy2_w)
            else:
                # Append to master arrays
                #append_array(master=lnPi,newdata=lnPi_w,smooth_type="mean")
                Nold = [x for x in N]
                N, lnPi = append_data(N,lnPi,N_w,lnPi_w,smooth_type="mean")
                Ntmp, energy = append_data(Nold,energy,N_w,energy_w,smooth_type="mean")
                #Ntmp, energy2 = append_data(Nold,energy2,N_w,lnPi_w,smooth_type="mean")
        
        #Convert to NumPy Arrays
        lnPi = np.array(lnPi)
        energy = np.array(energy)
        
        # Normalize lnPi
        lnPi = lnPi - max(lnPi)
        lnPi = lnPi - np.log(sum(np.exp(lnPi)))
        
        # Adjust Energy so that E(Nmin) = 0
        #energy2_master = energy2_master - 2.*energy_master[0]*energy_master + energy_master[0]**2
        energy = energy - energy[0]
        
        # Convert lnPi to format required by lnPi Class
        lnPi_data = np.array([ [float(Ni), lnPii] for Ni,lnPii in zip(N,lnPi) ])
        
        #Build class object using "from_data" method in parent class
        child = cls.from_data(lnPi_data,
                              mu=lnZ/beta,
                              volume=volume,
                              beta=beta,
                              num_phases_max=2,
                              argmax_kwargs=dict(min_distance=[5,10,20,40]),
                              ftag_phases=tag_phases)
        child.energy = np.array(energy)
        
        return child

    # Override the base class reweight method because we need the returned object to carry sublcass attributes
    #  NOTE: If it is necessary to derive a class *from this derived class*, then a new reweight method
    #        will be needed
    def reweight(self,
                 muRW,
                 ZeroMax=True,
                 Pad=False,
                 **kwargs):
        # Override the base class method because we need the returned object to carry subclass attributes
        child = self.copy(
            base=self.base.reweight(muRW, ZeroMax=ZeroMax, Pad=Pad, **kwargs),
            phases='get',
            argmax='get')
        # Copy new_class attributes to child object
        child.energy = copy.deepcopy(self.energy)
        #child.energy2 = copy.deepcopy(self.energy2)
        child.Sx = copy.deepcopy(self.Sx)
        child.canSx = copy.deepcopy(self.canSx)
        return child
    
    def extrapolate(self,
                    beta_extrap,
                    Normalize=False,
                    CheckTail=False,
                    CheckExtrap=True)
    
        if self.extrapolated and CheckExtrap:
            raise AttributeError('Extrapolating already *extrapolated* lnPi is dangerous. Disable via the CheckExtrap=False flag')
    
        try:
            if self.energy2.shape != self.base.data.shape:
                raise AttributeError('Canonical U^2 array is wrong size')
        except:
            raise AttributeError('Extrapolation requires input of <U^2>')
        
        child.canSx = False # Force recalculation of the canonical entropy
        child.Sx = np.array([]) ## HOW DO WE SHAPE THIS?
        child.extrapolated = True
    
    return child
    
    # PROPERTIES
    #FIGURE OUT HOW TO USE WPK'S CACHE OPS TO SAVE TIME HERE
    
    @property
    def Uaves(self):
        try:
            if self.energy.shape != self.base.data.shape:
                raise AttributeError('Canonical energy array is wrong size')
        except:
            raise AttributeError('User must input canonical energy before requesting average energy')
        Uavg = np.array([(x.pi_norm * self.energy).reshape(x.ndim,-1).sum(axis=-1).data
                for x in self])
        return Uavg
    
    @property
    def canonical_Sx(self):
        try:
            if self.energy.shape != self.base.data.shape:
                raise AttributeError('Canonical energy array is wrong size')
        except:
            raise AttributeError('User must input canonical energy before requesting canonical entropy')
        Sx = self.base.data - self.base.data[0] + self.base.beta*self.energy \
             - self.base.beta*self.base.mu[0]*self.base.coords
        self.Sx = Sx
        self.canSx = True
        return Sx
    
    @property
    def grand_canonical_Sx(self):
        if not self.canSx:
            self.Sx = self.canonical_Sx
        Sx_avg = np.array([(x.pi_norm * self.Sx).reshape(x.ndim,-1).sum(axis=-1).data
                for x in self])
        return Sx_avg
    
source = "/home/local/NIST/dsideriu/Workstation_Research/Monte_Carlo/my_FEASST/multiprocessing/overlap_4/tmp/"
stub = "rst_core"

new = feasst_analysis.from_restart(source,stub)
#print(new)
print(new.mu)
#print(new.base.data)

#fig = plt.figure(figsize=(8,8))
#plt.plot(new.base.coords[0],new.base.data)
#plt.show()

#fig = plt.figure(figsize=(8,8))
#plt.plot(new.base.coords[0],new.energy)
#plt.show()

print(new.Naves)
print(new.Uaves)
#print(new.canonical_Sx)
print(new.grand_canonical_Sx)
print()


new2 = new.reweight(new.mu-0.2)
print(new2.Naves)
print(new2.Uaves)
#print(new.canonical_Sx)
print(new2.grand_canonical_Sx)

SyntaxError: invalid syntax (<ipython-input-44-230e6fe7be4e>, line 160)